In [1]:
import os
import sys
import pandas as pd
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity

ROOT = os.path.abspath(os.path.join(os.getcwd()))
if ROOT not in sys.path:
    sys.path.append(ROOT)

import scripts.auxiliary_functions as af


In [2]:
# Octopaminergic cell types from fafb, mcns, and manc.
OA_fafb = ["OA-ASM1", "OA-ASM2", "OA-ASM3", "SMP143", "SMP142", "OA-AL2i1", "OA-AL2i2", "OA-AL2i3", "OA-AL2i4", "OA-AL2b1", "DNg34", "DNg104", "OA-VPM3", "OA-VPM4", "OA-VUMa1", "OA-VUMa2", "OA-VUMa3", "OA-VUMa4", "OA-VUMa5", "OA-VUMa6", "OA-VUMa8", "DNge138", "DNge149", "DNge152", "DNg66", "DNge150", "DNge151", "DNd02", "DNd03"]
OA_mcns = ["OA-ASM1", "OA-ASM2", "OA-ASM3", "SMP143", "SMP142", "SMP145", "OA-AL2i1", "OA-AL2i2", "OA-AL2i3", "OA-AL2i4", "LoVCLo3", "DNg34", "DNg104", "OA-VPM3", "OA-VPM4", "OA-VUMa1", "OA-VUMa2", "OA-VUMa3", "OA-VUMa4", "OA-VUMa5", "OA-VUMa6", "OA-VUMa8", "DNge138", "DNge149", "DNge152", "DNg66", "DNge150", "DNge151", "DNd02", "DNd03"]
OA_manc = ["DNg34", "DNg104", "DNxn156", "DNxn158", "DNxl126", "DNut059", "DNg66", "DNxn103", "DNad005", "DNd02", "DNd03"]

In [3]:
# Create connectivity matrices for all datasets.
matrices = {
    "fafb": af.get_connectivity_matrix(cell_types=OA_fafb, dataset="fafb"),
    "mcns": af.get_connectivity_matrix(cell_types=OA_mcns, dataset="mcns"),
    "manc": af.get_connectivity_matrix(cell_types=OA_manc, dataset="manc"),
}

In [4]:
# Store connectivity matrices containing combined upstream+downstream connections, only upstream connections, and only downstream connections.
def get_matrix_dictionary(mat):
    return {
        "combined": mat,
        "upstream": mat.loc[:, mat.columns.get_level_values("direction") == "upstream"],
        "downstream": mat.loc[:, mat.columns.get_level_values("direction") == "downstream"],
    }

matrix_dicts = {direction: get_matrix_dictionary(mat) for direction, mat in matrices.items()}

In [5]:
# Get cosine similarity matrix from connectivity matrix.
def get_connectivity_similarity(matrix_dicts, dataset, direction="combined"):
    mat = matrix_dicts[dataset][direction]
    X = mat.values

    if dataset not in matrix_dicts:
        raise ValueError(f"Dataset must be one of {list(matrix_dicts)}.")

    if direction not in matrix_dicts[dataset]:
        raise ValueError("Direction must be 'combined', 'upstream', or 'downstream'.")

    cos_sim = cosine_similarity(X)

    return pd.DataFrame(cos_sim, index=mat.index, columns=mat.index)

# We will get the similarity matrix for FAFB OA neurons, based on both their concatenated upstream and downstream connections, as an example.
cos_df = get_connectivity_similarity(
    matrix_dicts,
    dataset="fafb",
    direction="combined"
)

# As a second example, this would be how to get the similarity matrix for only the upstream connections of MCNS OA neurons.
# cos_df = get_connectivity_similarity(
#     matrix_dicts,
#     dataset="mcns",
#     direction="upstream"
# )

In [6]:
# Get heatmap showing cosine similarity between individual cells.
fig = px.imshow(
    cos_df,
    x=[af.get_label(id_) for id_ in cos_df.columns],
    y=[af.get_label(id_) for id_ in cos_df.index],
    color_continuous_scale="Oranges",
    zmin=0,
    zmax=1,
    aspect="equal",
    height=2000,
    width=2000,
)

fig.show()